Creation of the notebook

In [25]:
import numpy as np # Used to handle data in a vectorized manner
import pandas as pd # Used for data analsysis
import requests # Used for webscraping



Web page scraping

In [26]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

In [27]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [28]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

In [29]:
table = soup.find('table',{'class':'wikitable sortable'})

In [30]:
table_rows = table.find_all('tr')

In [31]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

Transformation of data into pandas dataframe

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [33]:
df.shape

(287, 3)

In [34]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()] 

In [35]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

Re-indexing of the Dataframe

In [36]:
df1 = df.reset_index()

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
index            210 non-null int64
PostalCode       210 non-null object
Borough          210 non-null object
Neighbourhood    210 non-null object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


In [38]:
df1.shape

(210, 4)

In [47]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

In [48]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [49]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [50]:
df3 = df2.reset_index()

In [51]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [52]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [53]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


The number of rows of your dataframe

In [54]:
df3.shape

(103, 3)